In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import os
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


# Growth Accounting

In [ ]:
select
    o_custkey as id,
    date_trunc(year, o_orderdate) as order_year,
    sum(o_totalprice) as total
from SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS
group by all
order by id, order_year

In [ ]:
df = cell3.to_pandas()

#pivot data to add row for each id:year with no revenue
result = df.pivot_table(
    index='ID',
    columns='ORDER_YEAR', 
    values='TOTAL',
    fill_value=0
).reset_index().melt(
    id_vars='ID',
    var_name='ORDER_YEAR',
    value_name='TOTAL'
)

# save the dataframe as table for SQL querying 
df = session.create_dataframe(result)
df.write.mode("overwrite").save_as_table("df", table_type="temporary")

In [ ]:
select * from df
order by id, order_year
limit 10

In [ ]:
with windowed as (
    
    select
        *,
        sum(total) over(partition by id order by order_year asc) as lifetime_spend,
        coalesce(lag(total) over(partition by id order by order_year asc), 0) as previous_year_total,
    from df

)

select *,
  case
    when total = previous_year_total and total > 0 then 'retained'
    when total > 0 and previous_year_total = 0 and lifetime_spend = total then 'new'
    when total = 0 and previous_year_total > 0 then 'churned'
    when total > previous_year_total and previous_year_total > 0 then 'expanded'
    when total < previous_year_total and previous_year_total > 0 then 'contracted'
    when total > 0 and previous_year_total = 0 and lifetime_spend > total then 'resurrected'
  else 'irrelevant' end as category,
  case category
    when 'retained' then 0
    when 'new' then total
    when 'churned' then (-1 * previous_year_total)
    when 'expanded' then total - previous_year_total
    when 'contracted' then (-1 * (previous_year_total - total))
    when 'resurrected' then total
  else 0 end as net_change
from windowed
order by id, order_year

In [ ]:
select
    date_part(year, order_year) as order_year,
    category,
    round(sum(total)) as total,
    round(sum(net_change)) as net_change
from {{ cell6 }}
group by all

In [ ]:
# Option to define dictionary to color code each category, may need to use matplotlib
# Option to use altair for better control of ticks on Y axis
st.bar_chart(cell4, x='ORDER_YEAR', y='NET_CHANGE', color='CATEGORY', height=750)

In [ ]:
df = cell6.to_pandas()
button_csv = df.to_csv().encode("utf-8")
st.download_button(label="Download", data=button_csv, file_name="growth_accounting.csv", mime="text/csv")

# Forecasting

# API Enrichment

In [ ]:
import requests

def get_wiki_extract(title):
    # Base URL for Wikipedia's API
    url = "https://en.wikipedia.org/w/api.php"
    
    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "exintro": True,  # Only get the intro section
        "explaintext": True,  # Get plain text instead of HTML
    }
    
    # Make the request
    response = requests.get(url, params=params)
    
    # Check if request was successful
    if response.status_code == 200:
        data = response.json()
        # Navigate through the JSON response to get the extract
        pages = data["query"]["pages"]
        # Get the first (and only) page's extract
        page = list(pages.values())[0]
        return page.get("extract", "No extract available")
    else:
        return f"Error: {response.status_code}"

cat_breeds = [
    'Abyssinian_cat',
    'Aegean_cat',
    'American_Bobtail',
    'American_Curl',
    'American_Ringtail',
    'American_Shorthair',
    'American_Wirehair',
    'Arabian_Mau',
    'Asian_cat',
    'Asian_Semi-longhair',
    'Australian_Mist',
    'Balinese_cat',
    'Bambino_cat',
    'Bengal_cat',
    'Birman',
    'Bombay_cat',
    'Brazilian_Shorthair',
    'British_Longhair',
    'British_Shorthair',
    'Burmese_cat',
    'Burmilla',
    'California_Spangled',
    'Chantilly-Tiffany',
    'Chartreux',
    'Chausie',
    'Colorpoint_Shorthair',
    'Cornish_Rex',
    'Cymric_cat',
    'Cyprus_cat',
    'Devon_Rex',
    'Donskoy_cat',
    'Dragon_Li',
    'Egyptian_Mau',
    'European_Shorthair',
    'Exotic_Shorthair',
    'Foldex_cat',
    'German_Rex',
    'Havana_Brown',
    'Highlander_cat',
    'Himalayan_cat',
    'Japanese_Bobtail',
    'Javanese_cat',
    'Kanaani_cat',
    'Khao_Manee',
    'Kinkalow',
    'Korat',
    'Korean_Bobtail',
    'Kurilian_Bobtail',
    'Lambkin_cat',
    'LaPerm',
    'Lykoi',
    'Maine_Coon',
    'Manx_cat',
    'Mekong_Bobtail',
    'Minskin',
    'Minuet_cat',
    'Munchkin_cat',
    'Nebelung',
    'Neva_Masquerade',
    'Norwegian_Forest_cat',
    'Ocicat',
    'Ojos_Azules',
    'Oriental_bicolour',
    'Oriental_Longhair',
    'Oriental_Shorthair',
    'Persian_cat',
    'Traditional_Persian',
    'Peterbald',
    'Pixie-bob',
    'Ragamuffin_cat',
    'Ragdoll',
    'Raas_cat',
    'Russian_Blue',
    'Savannah_cat',
    'Scottish_Fold',
    'Selkirk_Rex',
    'Serengeti_cat',
    'Siamese_cat',
    'Siberian_cat',
    'Singapura_cat',
    'Snowshoe_cat',
    'Sokoke',
    'Somali_cat',
    'Sphynx_cat',
    'Suphalak',
    'Thai_cat',
    'Tonkinese_cat',
    'Toybob',
    'Toyger',
    'Turkish_Angora',
    'Turkish_Van',
    'Van_cat',
    'Ukrainian_Levkoy',
    'York_Chocolate'
]
csv_list = []

for cat in cat_breeds:
    print(cat)
    extract = get_wiki_extract(cat)
    print(extract)
    csv_list.append((cat, extract))

# Convert to dataframe and save
df = pd.DataFrame(csv_list, columns=['breed', 'description'])
df.to_csv('cat_breeds.csv', index=False, encoding='utf-8')